In [ ]:
import math
import numpy as np
import pandas as pd
import os
from google.colab import drive
import time
import sys
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
EZlist = [1,2,3,4]
FInput = '/content/drive/MyDrive/Research/Freight/TourSetGen/TinyEM/Tiny Freight.csv' ##These can also be manually altered in the cell below for easier testing
FIn = pd.read_csv(FInput) 
FFreightIn = FIn['Weight Attr'] ## Total Freight originating at a zone per commod 
OInput = '/content/drive/MyDrive/Research/Freight/TourSetGen/TinyEM/Tiny Trucks.csv'
OIn = pd.read_csv(OInput) 
OTrucksIn = OIn['Trucks Pro'] ##Total Trucks 
h = 10
L = 3  

In [ ]:
FFreightIn.iloc[0]=400
FFreightIn.iloc[1]=300
FFreightIn.iloc[2]=200
FFreightIn.iloc[3]=100
OTrucksIn.iloc[0]=40
OTrucksIn.iloc[1]=10
OTrucksIn.iloc[2]=20
OTrucksIn.iloc[3]=30 
print(FFreightIn)
print(OTrucksIn)
print(OTrucksIn.sum())
print(FFreightIn.sum())

0    400
1    300
2    200
3    100
Name: Weight Attr, dtype: int64
0    40
1    10
2    20
3    30
Name: Trucks Pro, dtype: int64
100
1000


In [ ]:
##This cell is reserved for creating a/b + fm
def makeAB(M):  
  stopsIndex = 0
  la = (np.zeros([len(EZlist),len(M.index)]))
  lb = (np.zeros([len(EZlist),len(M.index)]))

  for m in M.index:
    for stops in M.columns:
      stopsIndex = stopsIndex + 1
      try: 
        if stopsIndex == 1: ##the first node is the Prod, everything else is Attr 
            la[int(float(M.iloc[m][stops]))][m] = 1
        else:          
            lb[int(float(M.iloc[m][stops]))][m] = 1
      except:
          pass
    stopsIndex = 0
  return la,lb

def makef(M):
  f = []
  for m in S:
    beta = 1/L
    sm = S[m]
    f.append(math.exp(-1*beta*sm))

  return f

In [ ]:
def yFirst(y1):
  denomHolder = []
  Denom = []
  for j in range(len(FFreightIn)):
    for m in range(len(S)):
      denomHolder.append(f[m]*b[j][m])
    Denom.append( 1/sum(denomHolder))
    denomHolder.clear()
  for m in range(len(S)):
    for j in range(len(FFreightIn)):
      y1[j][m] = FFreightIn[j] * (b[j][m] * f[m]) * Denom[j]
  
  for j in range(len(FFreightIn)):
    print(FFreightIn[j]-np.sum(y1,axis=1)[j])
  print(sum(sum(y1)))
  return y1

def tFirst(t1,y1,hk):
  yHolder = []
  for m in range(len(S)):
    for j in range(len(FFreightIn)):
      yHolder.append(y1[j][m])
    t1[m] = sum(yHolder)/hk
    yHolder.clear()     

  print("t1",sum(t1),"h",hk,"division",np.sum(y1)/hk) 
  return t1

In [ ]:
def Balance(tIn,yIn):
  iterCounter = 0
  
  tBal = np.zeros_like(tIn)
  yBal = np.zeros_like(yIn)
  tBalN = np.zeros_like(tBal)
  yBalN = np.zeros_like(yBal)
  
  ##Update t 
  #construct denom first
  denomHolderT = []
  denomBalT = []
  for i in range(len(OTrucksIn)):
    for m in range(len(S)):
      denomHolderT.append(a[i][m]*tIn[m])
    denomBalT.append(1/sum(denomHolderT))
    denomHolderT.clear()

  tHolder = []
  for m in range(len(S)):
    for i in range(len(OTrucksIn)):
      tHolder.append(OTrucksIn[i] * a[i][m] * tIn[m] * denomBalT[i])
    tBal[m] = sum(tHolder)
    tHolder.clear() 
  
  ##Update y
  #construct denom first
  denomHolderY = []
  denomBalY = []
  for j in range(len(FFreightIn)):
    for m in range(len(S)):
      denomHolderY.append(b[j][m]*( tBal[m] / sum(yIn)[m] )*yIn[j][m])
    denomBalY.append(1/sum(denomHolderY))
    denomHolderY.clear()

  for j in range(len(FFreightIn)):
    for m in range(len(S)):
      yBal[j][m] = FFreightIn[j] *  b[j][m] * (tBal[m]/sum(yIn)[m]) * yIn[j][m] * denomBalY[j] 

  delT = []
  delY = []

  #Calculate Improvment
  for m in range(len(S)):
    delT.append(abs(tBal[m] - tIn[m]))

  #Iterate if Fail Tours Tolerance Check
  if abs(sum(delT)) > .01:
    print("tBalCall",delT)
    delT.clear()
    tBal,yBal = Balance(tBal,yBal)
  
  for m in range(len(S)):
    for j in range(len(FFreightIn)):
      delY.append(abs(yBal[j][m] - yIn[j][m]))

  #Iterate if Fail Freight Tolerance Check
  if abs(sum(delY)) > .005:
    delY.clear( )
    tBal,yBal = Balance(tBal,yBal)
    
  return tBal, yBal

In [ ]:
timeHolder =[]
nTours = [8]
kIter = 0
for nT in nTours:
  timeHolder.append(time.time())
  MInput = '/content/drive/MyDrive/Research/Freight/TourSetGen/TinyEM/Tiny Tours.csv'
  MTours = pd.read_csv(MInput) ## Tour Set list 
  MTours = MTours.drop(['Unnamed: 0'],axis=1)
  S = list(MTours.count(axis=1))
  a,b = makeAB(MTours)
  f = makef(MTours)

  t0 = np.zeros(len(MTours.index))
  y0 = np.zeros((len(EZlist),len(MTours.index)))

  y1 = yFirst(y0)
  t1 = tFirst(t0,y1,h)  


  tFin,yFin = Balance(t1,y1)
  
  timeHolder.append(time.time())
  diffTime = timeHolder[-1]-timeHolder[-2]
  kIter = kIter +1
 

print("TLIB Done!") 

0.0
0.0
0.0
0.0
1000.0
t1 100.0 h 10 division 100.0
tBalCall [4.772727272727273, 4.166666666666666, 12.857142857142858, 1.1538461538461533, 9.642857142857142, 8.333333333333332, 2.7272727272727275, 1.3461538461538467]
TLIB Done!


In [ ]:
print(t1)
print(np.around(tFin),sum(tFin))
print(y1)
print(np.around(yFin),sum(sum(yFin)))
print("vert sum",sum(np.around(yFin))/10)
print("horz sum",np.sum( np.around(yFin),  axis=1 ))
print(diffTime)

[17.5  7.5 10.  15.   7.5 15.  10.  17.5]
[13.  3. 23. 14. 17.  7.  7. 16.] 100.0
[[100.   0.   0. 100.   0. 100.   0. 100.]
 [ 75.   0.  75.   0.   0.   0.  75.  75.]
 [  0.  50.   0.  50.  50.  50.   0.   0.]
 [  0.  25.  25.   0.  25.   0.  25.   0.]]
[[102.   0.   0. 113.   0.  55.   0. 130.]
 [ 25.   0. 185.   0.   0.   0.  59.  32.]
 [  0.  26.   0.  25. 136.  12.   0.   0.]
 [  0.   7.  44.   0.  35.   0.  14.   0.]] 1000.0000000000001
vert sum [12.7  3.3 22.9 13.8 17.1  6.7  7.3 16.2]
horz sum [400. 301. 199. 100.]
0.24889278411865234


In [ ]:
print(a)
print(b)
#print(OTrucksIn)
#print(FFreightIn)

[[0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1.]]
[[1. 0. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0. 1. 1.]
 [0. 1. 0. 1. 1. 1. 0. 0.]
 [0. 1. 1. 0. 1. 0. 1. 0.]]


In [ ]:
print(sum(sum(y1)))
print(sum(sum(yFin)))
print(sum(t1))
print(sum(tFin))

1000.0
1000.0000000000001
100.0
100.0
